In [1]:
# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 7/7 [01:05<00:00,  9.37s/it]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

In [5]:
# 使用原始 ChatGLM3-6B 模型
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是《易经》中的第一卦，也是八卦之一。乾卦的卦象是由三个阳爻夹一个阴爻构成，象征着天、阳气、强盛、刚健等特质。在《易经》中，乾卦代表着一种积极向上、刚强有力的精神面貌。

乾卦的含义非常丰富，它既象征着阳刚之气、壮丽、威严等特质，也象征着事物发展变化的规律。乾卦所传达的信息是，事物发展到一定程度就会走向鼎盛，但同时也要注意防止过度刚强，以免引起物极必反。

在《易经》的六十四卦中，乾卦经常与其他卦相组合，以表达更复杂的含义。例如，乾卦与坤卦组合成乾坤之卦，象征着天地之间的和谐与配合；与震卦组合成震乾之卦，象征着激发内在的刚强力量等。因此，乾卦在《易经》中具有重要的象征意义，是研究和理解《易经》不可或缺的一部分。


In [8]:
print(history)

[{'role': 'user', 'content': '解释下乾卦是什么？'}, {'role': 'assistant', 'metadata': '', 'content': '乾卦是《易经》中的第一卦，也是八卦之一。乾卦的卦象是由三个阳爻夹一个阴爻构成，象征着天、阳气、强盛、刚健等特质。在《易经》中，乾卦代表着一种积极向上、刚强有力的精神面貌。\n\n乾卦的含义非常丰富，它既象征着阳刚之气、壮丽、威严等特质，也象征着事物发展变化的规律。乾卦所传达的信息是，事物发展到一定程度就会走向鼎盛，但同时也要注意防止过度刚强，以免引起物极必反。\n\n在《易经》的六十四卦中，乾卦经常与其他卦相组合，以表达更复杂的含义。例如，乾卦与坤卦组合成乾坤之卦，象征着天地之间的和谐与配合；与震卦组合成震乾之卦，象征着激发内在的刚强力量等。因此，乾卦在《易经》中具有重要的象征意义，是研究和理解《易经》不可或缺的一部分。'}]


In [9]:
# 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是《易经》中的第五卦，它的卦象是由两个阴爻夹一个阳爻构成，象征着事物之间的矛盾、争端、诉讼等。因此，讼卦的含义主要涉及到人际关系、法律、争端、战争等方面的问题。

在《易经》中，讼卦代表着一种平等、公正的态度，它提醒人们在处理矛盾和争端时，应该保持公正客观，避免偏袒某一方，以达到和谐稳定的状态。

讼卦也象征着事物发展变化的规律，它告诉我们，任何事物在发展过程中都会遇到矛盾和争端，但是只要我们能够以公正客观的态度去处理这些问题，就能够找到解决问题的方法，使事物得以顺利发展。

因此，讼卦在《易经》中具有重要的象征意义，是研究和理解《易经》不可或缺的一部分。


In [11]:
# 使用微调后的 ChatGLM3-6B
# 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
# ./models/THUDM/chatglm3-6b-epoch3-20250826_092818
timestamp = "20250826_092818"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [12]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

In [13]:
# 微调前后效果对比
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦，是周易中的一卦，由两个乾卦叠加而成，代表天地的正义和刚健。在这个卦中，预示着君子观者，卦象显示了天地的刚健和正派。

乾卦的卦象由六个阳爻组成，位于卦象的北部，代表君子。观者，君子观此卦象，感到天空广博、云彩高洁，心情变得宽广大大。因此，君子拟定了高明的君子 immur（pick up the ground），即"高")，来适应天空，展现君子的高尚品质。

乾卦的卦象预示着天地的刚健和正派，代表着君子刚健的品质和坚定不屈的决心。在事业上，象征着君子刚健果敢， urgency，需要坚定决心，积极行动。在经商中，象征着商贾们刚健泉涌，大富大贵。在求名，表示君子们品德高尚，声誉高远。

总体来说，乾卦是事业经商中的吉祥之卦，也是求名的正面之卦。然而，虽然卦象吉利，但并非完全有利于个人，需要根据实际情况来选择方案。对于经商和求名，需要坚定正义感，积极行动，以便获得美好的结果。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250826_092818）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [14]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
{'name': '并要求对方用乾刚来相对应。如果刚卦与刚卦相待，则表示刚柔相济，刚中有柔，柔中有刚的平衡之象，这是一个吉祥的征兆。然而，如果动乱之情起自争卦，那么动乱之灾将无法避免。对于经商而言，经商之前应仔细规划，求得合理的经营策略，动乱之灾 inline-text-align:center;', 'content': "\n初六，始得愿fully，求得 first-rate 通晓，开始得到私欲，宜顺水推舟。初六纯阳，象征开始时 lies in the hands of the innocent and pure, who will eventually be rewarded.\n\n六二，得阴后脱，脱险得得，渡河拆桥 \n\n六三，水浅沙埋，困境难出，风险四伏，宜速寻解决办法。\n\n六四，entials 外，宜勇于 Indulging in luxury, but beware of risks, especially in business.\n\n六五，弦池之池，得志如登，如鱼得水，乐正无穷。\n\n六六，马到成功，功成得志，如镜破烂，万事具备。\n\n然而， these days, more and more people are seeking happiness and success in life, but the path to happiness and success is often long and difficult, and it's not always possible to achieve our goals right away. Therefore, it's important to be patient, persistent, and adaptable, and to embrace the changes and challenges that life presents. The outcome of the query depends on the rightousness and purity of the heart, as well as the courage and determination to overcome difficulties and ob

In [15]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250826_092818）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。
